In [16]:
import os


def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable."
    it = iter(seq)
    result = tuple(islice(it, n))

    if len(result) == n:
        yield result    

    for elem in it:
        result = result[1:] + (elem,)
        yield result

        
def Input(day):
    """Fetch the data input from disk or URL."""
    filename = os.path.join('../data/advent2017/input{}.txt'.format(day))
    return open(filename)

# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

In [18]:
def sum_consecutive(data):
    nums = list(map(int, data))
    total = 0
    for index, n in enumerate(nums):
        next_n = nums[(index + 1) % len(nums)] 
        if n == next_n:
            total += n
    return total


assert sum_consecutive('1122') == 3
assert sum_consecutive('1111') == 4
assert sum_consecutive('1234') == 0
assert sum_consecutive('91212129') == 9

sum_consecutive(Input(1).read().strip())

1182

In [21]:
def sum_half(data):
    nums = list(map(int, data))
    total = 0
    jump_distance = len(nums) // 2
    for index, n in enumerate(nums):
        next_n = nums[(index + jump_distance) % len(nums)] 
        if n == next_n:
            total += n
    return total
    

assert sum_half('1212') == 6
assert sum_half('1221') == 0
assert sum_half('123425') == 4
assert sum_half('123123') == 12
assert sum_half('12131415') == 4

sum_half(Input(1).read().strip())

1152